## Данные:

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

##### Installation

In [2]:
!apt-get install bedtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
bedtools is already the newest version (2.26.0+dfsg-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


##### blacklisted

In [3]:
#blacklisted
#!wget https://www.encodeproject.org/files/ENCFF001TDO/@@download/ENCFF001TDO.bed.gz
#!gunzip ENCFF001TDO.bed.gz
#!cut --complement -f4,5,6 ENCFF001TDO.bed > output.bed

In [4]:
#!sort -k 1,1 -k2,2n -o output.bed output.bed
#!bedtools merge -i output.bed > blacklisted.bed

##### ZDNA

In [5]:
#!wget https://raw.githubusercontent.com/adriadar/cw/main/data/DeepZ.bed
#!wget https://raw.githubusercontent.com/adriadar/cw/main/data/zhunt.bed

##### Genome

In [6]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/mm10/bigZips/chromFa.tar.gz

--2022-06-07 10:01:14--  https://hgdownload.soe.ucsc.edu/goldenPath/mm10/bigZips/chromFa.tar.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870152350 (830M) [application/x-gzip]
Saving to: ‘chromFa.tar.gz.1’

chromFa.tar.gz.1    100%[===================>] 829.84M  39.1MB/s    in 22s     

2022-06-07 10:01:36 (37.9 MB/s) - ‘chromFa.tar.gz.1’ saved [870152350/870152350]



In [7]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes

--2022-06-07 10:01:36--  https://hgdownload.soe.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1405 (1.4K)
Saving to: ‘mm10.chrom.sizes.1’

mm10.chrom.sizes.1  100%[===================>]   1.37K  --.-KB/s    in 0s      

2022-06-07 10:01:37 (121 MB/s) - ‘mm10.chrom.sizes.1’ saved [1405/1405]



In [8]:
!mkdir chromFA
!tar -zxvf chromFa.tar.gz -C chromFA

mkdir: cannot create directory ‘chromFA’: File exists
chr1.fa
chr10.fa
chr11.fa
chr12.fa
chr13.fa
chr14.fa
chr15.fa
chr16.fa
chr17.fa
chr18.fa
chr19.fa
chr1_GL456210_random.fa
chr1_GL456211_random.fa
chr1_GL456212_random.fa
chr1_GL456213_random.fa
chr1_GL456221_random.fa
chr2.fa
chr3.fa
chr4.fa
chr4_GL456216_random.fa
chr4_GL456350_random.fa
chr4_JH584292_random.fa
chr4_JH584293_random.fa
chr4_JH584294_random.fa
chr4_JH584295_random.fa
chr5.fa
chr5_GL456354_random.fa
chr5_JH584296_random.fa
chr5_JH584297_random.fa
chr5_JH584298_random.fa
chr5_JH584299_random.fa
chr6.fa
chr7.fa
chr7_GL456219_random.fa
chr8.fa
chr9.fa
chrM.fa
chrUn_GL456239.fa
chrUn_GL456359.fa
chrUn_GL456360.fa
chrUn_GL456366.fa
chrUn_GL456367.fa
chrUn_GL456368.fa
chrUn_GL456370.fa
chrUn_GL456372.fa
chrUn_GL456378.fa
chrUn_GL456379.fa
chrUn_GL456381.fa
chrUn_GL456382.fa
chrUn_GL456383.fa
chrUn_GL456385.fa
chrUn_GL456387.fa
chrUn_GL456389.fa
chrUn_GL456390.fa
chrUn_GL456392.fa
chrUn_GL456393.fa
chrUn_GL456394.fa
chrUn_GL

## Preparation

In [9]:
import pandas as pd
import numpy as np
import os

In [10]:
path = "/content/chromFA/"
dir_list = os.listdir(path)

In [11]:
# соберем геном
for i in dir_list:
  file_name = f"chromFA/{i}"
  !cat $file_name >> genome.fa

In [12]:
# объединим zhunt и DeepZ
#!cat zhunt.bed DeepZ.bed > for_merge.bed
#!sort -k1,1 -k2,2n for_merge.bed > for_merge_sorted.bed
#!bedtools merge -i for_merge_sorted.bed > zdna.bed

In [13]:
chr_len = pd.read_csv('mm10.chrom.sizes', delimiter='\t', names=['chr', 'len'])
chr_len.head()

,chr,len
0,chr1,195471971
1,chr2,182113224
2,chrX,171031299
3,chr3,160039680
4,chr4,156508116


In [14]:
# соберем файл длинн и отсортируем
chr_len.to_csv('genome_chrom_len.txt', header=False, index=False, sep='\t')
!sort -k1,1 -k2,2n genome_chrom_len.txt > genome_chrom_len_sorted.txt

In [15]:
! rev curax14.narrowPeak | cut -d$'\t' -f 8- | rev > curax14.bed
! sort -k1,1 -k2,2n curax14.bed > curax14_sort.bed

In [16]:
! rev curax10.narrowPeak | cut -d$'\t' -f 8- | rev > curax10.bed
! sort -k1,1 -k2,2n curax10.bed > curax10_sort.bed

### Положительный класс

In [137]:
# загрузим полученные Z-DNA
zdna = pd.read_csv('curax14_sort.bed', sep='\t', comment='t', header=None)
header = ['chrom', 'start', 'end']
zdna.columns = header[:len(zdna.columns)]
zdna['len'] = zdna.end - zdna.start
zdna

,chrom,start,end,len
0,chr1,4559741,4560197,456
1,chr1,5072167,5072549,382
2,chr1,8029931,8030529,598
3,chr1,9177220,9177628,408
4,chr1,14975703,14976118,415
...,...,...,...,...
1646,chrY,90777622,90778737,1115
1647,chrY,90781178,90781485,307
1648,chrY,90784489,90784879,390
1649,chrY,90824558,90824842,284


In [18]:
#zdna = zdna[zdna.len >= 800]
#zdna.reset_index(drop=True, inplace=True)
#zdna

In [19]:
#zdna2 = zdna[zdna.len >= 1600]
#zdna2

In [20]:
#zdna = zdna[zdna.len < 1600]
#zdna

In [138]:
zdna3 = pd.DataFrame(columns=['chrom', 'start', 'end', 'len', 'cut'])
for index, row in zdna.iterrows():
  cur_st = row.start
  while cur_st <= row.end:
      if cur_st + 800 <= row.end:
          zdna3 = zdna3.append({'chrom':row.chrom, 'start':cur_st, 'end':cur_st + 800, 'len':800, 'cut':0}, ignore_index = True)
      else:
          zdna3 = zdna3.append({'chrom':row.chrom, 'start':cur_st, 'end':row.end, 'len':row.end - cur_st,'cut':1}, ignore_index = True)
      cur_st = cur_st + 800
zdna3

,chrom,start,end,len,cut
0,chr1,4559741,4560197,456,1
1,chr1,5072167,5072549,382,1
2,chr1,8029931,8030529,598,1
3,chr1,9177220,9177628,408,1
4,chr1,14975703,14976118,415,1
...,...,...,...,...,...
1771,chrY,90778422,90778737,315,1
1772,chrY,90781178,90781485,307,1
1773,chrY,90784489,90784879,390,1
1774,chrY,90824558,90824842,284,1


In [139]:
zdna3['cut'].to_csv("cut.csv", sep='\t', header=False, index=False)

In [140]:
zdna3.drop(columns=['cut'], inplace=True)

In [22]:
#zdna = zdna.append(zdna3)
#zdna.reset_index(drop=True, inplace=True)
#zdna

In [23]:
#zdna['centre_pos'] = (zdna.end - zdna.start) / 2
#zdna.centre_pos = zdna.centre_pos.astype(int)
#zdna['centre'] = zdna.start + zdna.centre_pos
#zdna

In [24]:
#zdna['new_start'] = zdna.centre - 400
#zdna['new_end'] = zdna.centre + 400
#zdna

In [141]:
zdna3.to_csv('new_zdna14.bed', index=False, header=False, sep='\t', columns=['chrom', 'start', 'end'])

In [142]:
!bedtools getfasta -fi genome.fa -bed new_zdna14.bed -fo zdna_seq14.fa

In [143]:
zdna_seq = pd.read_csv('zdna_seq14.fa', sep='\n', comment='>', header=None, names=['seq'])
zdna_seq

,seq
0,ctgccgaacttaggaaattagtctgaacaggtgagagggtgcgcca...
1,TCTGGCCGGTGTTTGCCATGCTCAGGCCGGTGTTTGCCATGCTCAG...
2,caaatctccaccaaattggattcacctgtatcctggtacacctgcg...
3,gaacacaaatgccagcccaggctactatacccggccaaactctcaa...
4,cggtcacccgtgcagcctgccctccgcggagtcccggagccaagaa...
...,...
1771,TCACAGTCATCCCCCTGCTTCAGAAAATCACTCACAGTCATCCCCC...
1772,GCCTCAGAAAATCACTCACAGTCATCCCCCTGCCTCAGAAAATCAC...
1773,cccgctctcttccccacAGGCTCGGCCGCGCACGGACGGGGCTTGC...
1774,acaggcagagactgggacagagacagatggacagaCCTGAGCTGCG...


In [144]:
zdna = zdna3

In [145]:
zdna['seq'] = zdna_seq['seq']

In [146]:
zdna

,chrom,start,end,len,seq
0,chr1,4559741,4560197,456,ctgccgaacttaggaaattagtctgaacaggtgagagggtgcgcca...
1,chr1,5072167,5072549,382,TCTGGCCGGTGTTTGCCATGCTCAGGCCGGTGTTTGCCATGCTCAG...
2,chr1,8029931,8030529,598,caaatctccaccaaattggattcacctgtatcctggtacacctgcg...
3,chr1,9177220,9177628,408,gaacacaaatgccagcccaggctactatacccggccaaactctcaa...
4,chr1,14975703,14976118,415,cggtcacccgtgcagcctgccctccgcggagtcccggagccaagaa...
...,...,...,...,...,...
1771,chrY,90778422,90778737,315,TCACAGTCATCCCCCTGCTTCAGAAAATCACTCACAGTCATCCCCC...
1772,chrY,90781178,90781485,307,GCCTCAGAAAATCACTCACAGTCATCCCCCTGCCTCAGAAAATCAC...
1773,chrY,90784489,90784879,390,cccgctctcttccccacAGGCTCGGCCGCGCACGGACGGGGCTTGC...
1774,chrY,90824558,90824842,284,acaggcagagactgggacagagacagatggacagaCCTGAGCTGCG...


In [31]:
#zdna_clear = []
#for index, row in zdna.iterrows():
#  if "N" not in row.seq:
#    zdna_clear.append([row.chrom, row.start, row.end, row.seq, row.len])
#zdna_clear = pd.DataFrame(zdna_clear, columns=['chrom', 'start', 'end', 'seq', 'len'])

In [32]:
#zdna.reset_index(drop=True, inplace=True)
#zdna_clear

In [33]:
#zdna_s.reset_index(drop=True, inplace=True)

In [147]:
def func(x):
    x[4] = x[4] + (800 - int(x[3])) * "N"
    return x

In [148]:
zdna_s = zdna.apply(func, axis=1)
zdna_s

,chrom,start,end,len,seq
0,chr1,4559741,4560197,456,ctgccgaacttaggaaattagtctgaacaggtgagagggtgcgcca...
1,chr1,5072167,5072549,382,TCTGGCCGGTGTTTGCCATGCTCAGGCCGGTGTTTGCCATGCTCAG...
2,chr1,8029931,8030529,598,caaatctccaccaaattggattcacctgtatcctggtacacctgcg...
3,chr1,9177220,9177628,408,gaacacaaatgccagcccaggctactatacccggccaaactctcaa...
4,chr1,14975703,14976118,415,cggtcacccgtgcagcctgccctccgcggagtcccggagccaagaa...
...,...,...,...,...,...
1771,chrY,90778422,90778737,315,TCACAGTCATCCCCCTGCTTCAGAAAATCACTCACAGTCATCCCCC...
1772,chrY,90781178,90781485,307,GCCTCAGAAAATCACTCACAGTCATCCCCCTGCCTCAGAAAATCAC...
1773,chrY,90784489,90784879,390,cccgctctcttccccacAGGCTCGGCCGCGCACGGACGGGGCTTGC...
1774,chrY,90824558,90824842,284,acaggcagagactgggacagagacagatggacagaCCTGAGCTGCG...


In [149]:
df_pos = zdna_s[['chrom', 'start', 'end', 'seq']].copy()

In [150]:
df_pos

,chrom,start,end,seq
0,chr1,4559741,4560197,ctgccgaacttaggaaattagtctgaacaggtgagagggtgcgcca...
1,chr1,5072167,5072549,TCTGGCCGGTGTTTGCCATGCTCAGGCCGGTGTTTGCCATGCTCAG...
2,chr1,8029931,8030529,caaatctccaccaaattggattcacctgtatcctggtacacctgcg...
3,chr1,9177220,9177628,gaacacaaatgccagcccaggctactatacccggccaaactctcaa...
4,chr1,14975703,14976118,cggtcacccgtgcagcctgccctccgcggagtcccggagccaagaa...
...,...,...,...,...
1771,chrY,90778422,90778737,TCACAGTCATCCCCCTGCTTCAGAAAATCACTCACAGTCATCCCCC...
1772,chrY,90781178,90781485,GCCTCAGAAAATCACTCACAGTCATCCCCCTGCCTCAGAAAATCAC...
1773,chrY,90784489,90784879,cccgctctcttccccacAGGCTCGGCCGCGCACGGACGGGGCTTGC...
1774,chrY,90824558,90824842,acaggcagagactgggacagagacagatggacagaCCTGAGCTGCG...


In [151]:
pos_seq = pd.DataFrame(df_pos.seq)
pos_seq

,seq
0,ctgccgaacttaggaaattagtctgaacaggtgagagggtgcgcca...
1,TCTGGCCGGTGTTTGCCATGCTCAGGCCGGTGTTTGCCATGCTCAG...
2,caaatctccaccaaattggattcacctgtatcctggtacacctgcg...
3,gaacacaaatgccagcccaggctactatacccggccaaactctcaa...
4,cggtcacccgtgcagcctgccctccgcggagtcccggagccaagaa...
...,...
1771,TCACAGTCATCCCCCTGCTTCAGAAAATCACTCACAGTCATCCCCC...
1772,GCCTCAGAAAATCACTCACAGTCATCCCCCTGCCTCAGAAAATCAC...
1773,cccgctctcttccccacAGGCTCGGCCGCGCACGGACGGGGCTTGC...
1774,acaggcagagactgggacagagacagatggacagaCCTGAGCTGCG...


In [106]:
pos_seq_t = pos_seq.sample(n = 300)
pos_seq_t

,seq
2326,ctcctgctcctgctgctcctcctgctcctcctgctcctgttcctcc...
471,CAGTCCTCCCACAACTCCTTGAGGTCATCAGTCAGTCCCCTCGGCC...
796,CAGGTCGTCGGCGCCCGGTGCCAGGTGCAGCGGCTCGGCGCGCAGC...
523,GCACCAGGCGCGGAAGCCTACCGGGTGCCCAGGGTGCAGCGCGATC...
3128,tgaccatgtgtgtgtatgtctatgtatgagtgtgtgtgtctgtgtg...
...,...
3345,cacctttttctgttttcctcgccatatttcacgtcctaaagtgtgt...
1517,tctgtgtgtgtgtgtctgtgtgtgtgtatgtgtctctgtgtgtgtg...
2449,CCCCCACTCACTCTGGCCCTCAAACTTGCGGATGATGGTGTCGAGG...
2029,GCAACCGCACTCCTCCTCAACTTCGAAGGCCGCACGCGCCCGGCTC...


### Отрицательный класс

def cut_segm(df):
    res = pd.DataFrame(columns=['chrom', 'start', 'end'])

    for key in chr_dict:
        arr = [0]
        d = df[df.chrom == key]

        for i, row in d.iterrows():
            if row.start == 0:
                arr.append(row.end - 1)
                continue
            if row.end == chr_dict[row.chrom]:
                arr.append(row.start)
                continue
            arr.append(row.start - 1)
            arr.append(row.end + 1)
        
        if arr[-1] != chr_dict[row.chrom]:
            arr.append(chr_dict[row.chrom])
        
        for i in range(0, len(arr), 2):
            res = res.append({'chrom' : key, 'start' : arr[i], 'end' : arr[i + 1]}, ignore_index=True)
    return res

whitelist = cut_segm(blacklist)
whitelist.head()

whitelist.to_csv('whitelisted.bed', index=False, header=False, sep='\t')

!head whitelisted.bed

In [39]:
chr_len["start"] = 0

In [40]:
chr_len = chr_len.reindex(columns=["chr", 'start', "len"])
chr_len.to_csv("full_genome_size.bed", sep='\t', header=False, index=False)

In [41]:
! bedtools subtract -a full_genome_size.bed -b curax10_sort.bed > not_zdna.bed

In [94]:
not_zdna = pd.read_csv('not_zdna.bed', sep='\t', comment='t', header=None)
header = ['chrom', 'start', 'end']
not_zdna.columns = header[:len(not_zdna.columns)]
not_zdna['len'] = not_zdna.end - not_zdna.start
not_zdna.head()

,chrom,start,end,len
0,chr1,0,3670918,3670918
1,chr1,3671737,4491828,820091
2,chr1,4492227,6441202,1948975
3,chr1,6441546,9545571,3104025
4,chr1,9545947,9635425,89478


In [95]:
not_zdna = not_zdna[not_zdna.len > 800]
not_zdna = not_zdna[not_zdna.len < 20000]
not_zdna

,chrom,start,end,len
70,chr1,171066184,171072418,6234
72,chr1,171073762,171080042,6280
89,chr2,5359240,5379073,19833
107,chr2,25159110,25161119,2009
118,chr2,28184653,28193032,8379
...,...,...,...,...
3313,chrUn_GL456396,7463,8607,1144
3316,chrUn_GL456396,15589,20787,5198
3319,chrM,9175,10074,899
3320,chr4_JH584292_random,0,14945,14945


In [70]:
not_zdna = not_zdna.sample(n = 100, random_state=0)
not_zdna

,chrom,start,end,len
2925,chrY,90738851,90740939,2088
499,chr3,109340905,117059376,7718471
2405,chr12,113738485,117153775,3415290
1249,chr7,65862603,66554456,691853
2215,chr13,46163372,46418862,255490
...,...,...,...,...
2136,chr11,101164429,102437472,1273043
1192,chr7,24482471,24533924,51453
3001,chr18,78048903,79109555,1060652
1816,chr14,65262426,65358479,96053


In [71]:
not_zdna = not_zdna.sort_index()

In [96]:
not_zdna.reset_index(drop=True, inplace=True)

In [79]:
not_zdna

,chrom,start,end,len
0,chr2,25159110,25161119,2009
1,chr2,58567635,58571267,3632
2,chr2,61812380,61815175,2795
3,chr2,98662727,98666571,3844
4,chr2,153528444,153529637,1193
...,...,...,...,...
123,chrUn_GL456390,21808,23890,2082
124,chrUn_GL456392,11731,15349,3618
125,chrUn_GL456396,7463,8607,1144
126,chrM,9175,10074,899


In [44]:
#whitelist['centre_pos'] = (whitelist.end - whitelist.start) / 2
#whitelist.centre_pos = whitelist.centre_pos.astype(int)
#whitelist['center'] = whitelist.start + whitelist.centre_pos
#whitelist

In [45]:
#whitelist['new_start'] = whitelist.center - 400
#whitelist['new_end'] = whitelist.center + 400
#whitelist.reset_index(drop=True, inplace=True)
#whitelist

In [97]:
not_zdna3 = pd.DataFrame(columns=['chrom', 'start', 'end', 'len'])
for index, row in not_zdna.iterrows():
  cur_st = row.start
  while cur_st <= row.end:
      if cur_st + 800 <= row.end:
          not_zdna3 = not_zdna3.append({'chrom':row.chrom, 'start':cur_st, 'end':cur_st + 800, 'len':800}, ignore_index = True)
      else:
          not_zdna3 = not_zdna3.append({'chrom':row.chrom, 'start':cur_st, 'end':row.end, 'len':row.end - cur_st}, ignore_index = True)
      cur_st = cur_st + 800
not_zdna3

,chrom,start,end,len
0,chr1,171066184,171066984,800
1,chr1,171066984,171067784,800
2,chr1,171067784,171068584,800
3,chr1,171068584,171069384,800
4,chr1,171069384,171070184,800
...,...,...,...,...
2640,chr4_JH584292_random,13600,14400,800
2641,chr4_JH584292_random,14400,14945,545
2642,chr4_JH584295_random,0,800,800
2643,chr4_JH584295_random,800,1600,800


In [100]:
not_zdna = not_zdna3
not_zdna3 = not_zdna3[["chrom", "start", 'end']]
not_zdna3

,chrom,start,end
0,chr1,171066184,171066984
1,chr1,171066984,171067784
2,chr1,171067784,171068584
3,chr1,171068584,171069384
4,chr1,171069384,171070184
...,...,...,...
2640,chr4_JH584292_random,13600,14400
2641,chr4_JH584292_random,14400,14945
2642,chr4_JH584295_random,0,800
2643,chr4_JH584295_random,800,1600


In [101]:
not_zdna3.to_csv("not_zdna_new.bed", sep='\t', header=False, index=False)

In [102]:
!bedtools getfasta -fi genome.fa -bed not_zdna_new.bed -fo not_zdna_seq.fa

In [103]:
not_zdna_seq = pd.read_csv('not_zdna_seq.fa', sep='\n', comment='>', header=None, names=['seq'])
not_zdna['seq'] = not_zdna_seq
not_zdna

,chrom,start,end,len,seq
0,chr1,171066184,171066984,800,GTCGGGGCTCACCCCAGGACCTCAGCTCGGGGCCCGGACTCTCCTA...
1,chr1,171066984,171067784,800,TATGCTTTCCCTTCTCCAtcttcttctccaaggagttatatatttg...
2,chr1,171067784,171068584,800,agagacagagagacagaggaagagagagacagagaggaagagagac...
3,chr1,171068584,171069384,800,tcacggatgattctccagccctggctgatctggaccacagattctt...
4,chr1,171069384,171070184,800,AGAGTTCATGAAGGAAAACAGCAGGTCCCTGCAACATTTGTCCTTA...
...,...,...,...,...,...
2640,chr4_JH584292_random,13600,14400,800,AGGTCACAGTTTGGTCTATCTGAGAGTTCATTGTATGGTCTATCTG...
2641,chr4_JH584292_random,14400,14945,545,GTGTGGTTTATCTGAATGCTCAGGATGTGGTCTATCTGAGTGGTCA...
2642,chr4_JH584295_random,0,800,800,GGCTGAGCGGTGACATCATGGGCGGCGGGGTCCCAGACAGGAAGTG...
2643,chr4_JH584295_random,800,1600,800,gtcagcgcaggagcaggaagcagggggcgtggcaggaagtgtgggg...


In [ ]:
#whitelist_clear = []
#for index, row in whitelist.iterrows():
#  if "N" not in row.seq:
#    whitelist_clear.append([row.chrom, row.new_start, row.new_end, row.seq])
#whitelist_clear = pd.DataFrame(whitelist_clear, columns=['chrom', 'start', 'end', 'seq'])

In [104]:
neg_df = pd.DataFrame(not_zdna.seq)
neg_df

,seq
0,GTCGGGGCTCACCCCAGGACCTCAGCTCGGGGCCCGGACTCTCCTA...
1,TATGCTTTCCCTTCTCCAtcttcttctccaaggagttatatatttg...
2,agagacagagagacagaggaagagagagacagagaggaagagagac...
3,tcacggatgattctccagccctggctgatctggaccacagattctt...
4,AGAGTTCATGAAGGAAAACAGCAGGTCCCTGCAACATTTGTCCTTA...
...,...
2640,AGGTCACAGTTTGGTCTATCTGAGAGTTCATTGTATGGTCTATCTG...
2641,GTGTGGTTTATCTGAATGCTCAGGATGTGGTCTATCTGAGTGGTCA...
2642,GGCTGAGCGGTGACATCATGGGCGGCGGGGTCCCAGACAGGAAGTG...
2643,gtcagcgcaggagcaggaagcagggggcgtggcaggaagtgtgggg...


In [107]:
neg_s = neg_df.sample(n = pos_seq_t.shape[0] * 5)
neg_s

,seq
382,TCAAAGATTGGGATGTTTTGAATGGGTTTTTCAAGAGAGGAAATAC...
776,tctgcgctgtggaaacactgctccccactcattccctgattggtca...
126,gccctgaactcaccatgtagcaaagggtgatcttgatctgaacttg...
245,tgtagaacatggtatttggccataaacattgaagaacatggaaaat...
662,tccttcctcccttcctccctcccttccttccttccttacttccttc...
...,...
1404,cacacccacacacccacacaccaacacccacccacacacactgaca...
402,aaccctcacaaacaccttgtatctctgactccactcaactctttgg...
2108,tctacaaagtgagttccaggacagccagggctacacagagaaactg...
2424,GCTCCCGGCCGCCCTAGGAGAGCGGGCGGAGTGGGTGTGGGTGTGC...


## сохраним файлы


In [108]:
pos_seq_t.to_csv('pos_class_m.csv', header=False, index=False)

In [153]:
pos_seq.to_csv('pos_class_m_full.csv', header=False, index=False)

In [152]:
zdna_s.to_csv('full_m.csv', header=False, index=False)

In [109]:
neg_s.to_csv('neg_class_m.csv',header=False, index=False)

In [110]:
from google.colab import files
files.download('pos_class_m.csv')
files.download('neg_class_m.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>